# mp4 파일 전송

In [ ]:
import socket
import time

server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
server.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
server.bind(('', 50000))

My_video = '/Users/winlab/Desktop/Summer.mp4'
video = open(My_video,'rb')
data = video.read()

port = 45000

header = data.find(b'mdat') + 5
l = len(data[header:])
n_h = 1
n_d = 1

def frag(N):
    n = 1
    while 1:
        if N/n >= 60000:
            n += 1
        else:
            return n

n_h = frag(header)
n_d = frag(l)

server.sendto(str(n_h).encode(), ('<broadcast>',port))
server.sendto(str(n_d).encode(), ('<broadcast>',port))

for i in range(n_h):
    server.sendto(data[i*header//n_h : (i+1)*header//n_h], ('<broadcast>',port))
    time.sleep(0.01)

for i in range(n_d):
    server.sendto(data[header + i*l//n_d : header + (i+1)*l//n_d], ('<broadcast>',port))
    time.sleep(0.01)

server.close()

# 인덱싱된 mp4 파일 전송

In [1]:
import numpy as np
import socket
import time

def indexing_mp4(v_path1, v_path2):
    with open(v_path1, 'rb') as v1:  # 첫번째 영상 바이너리 형식으로 읽어오기
        v1_data = v1.read()
        
    with open(v_path2, 'rb') as v2: # 두번째 영상 바이너리 형식으로 읽어오기
        v2_data = v2.read()
    
    data_idx1 = v1_data.find(b'mdat')
    header_idx1 = v1_data.find(b'moov')
    
    if data_idx1 < header_idx1:
        print("can't stream")
        return
    
    data_idx2 = v2_data.find(b'mdat')
    header_idx2 = v2_data.find(b'moov')
    
    if data_idx2 < header_idx2:
        print("can't stream")
        return
    
    np_v1 = np.frombuffer(v1_data, dtype='uint8') # 바이너리 해석
    np_v2 = np.frombuffer(v2_data, dtype='uint8')

    l1 = len(np_v1)
    l2 = len(np_v2)
    if l2 < l1:
        xor_len = l2
        add_data = np_v1[l2:l1]
    else:
        xor_len = l1
        add_data = np_v2[l1:l2]
        
    # 데이터의 길이를 비교
    
    xor_v = np.bitwise_xor(np_v1[:xor_len], np_v2[:xor_len]) # xor 연산
    xor_v_data = np.append(xor_v, add_data)

    xor_v_bin = xor_v_data.tostring() # 택 데이터 바이너리화
    return xor_v_bin, max(data_idx1, data_idx2) + 5

server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
server.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
server.bind(('', 50000))

v_path1 = '/Users/winlab/Desktop/Yes.mp4'
v_path2 = '/Users/winlab/Desktop/Summer.mp4'
stream_data, header = indexing_mp4(v_path1, v_path2)

My_video = '/Users/winlab/Desktop/indexing.mp4'
open(My_video, 'wb').write(stream_data)

video = open(My_video,'rb')
data = video.read()

l = len(data[header:])
n_h = 1
n_d = 1

def frag(N):
    n = 1
    while 1:
        if N/n >= 60000:
            n += 1
        else:
            return n

n_h = frag(header)
n_d = frag(l)

server.sendto(str(n_h).encode(), ('<broadcast>',45000))
server.sendto(str(n_d).encode(), ('<broadcast>',45000))
time.sleep(0.001)

for i in range(n_h):
    server.sendto(data[i*header//n_h : (i+1)*header//n_h], ('<broadcast>',45000))
    time.sleep(0.1)

for i in range(n_d):
    server.sendto(data[header + i*l//n_d : header + (i+1)*l//n_d], ('<broadcast>',45000))
    time.sleep(0.1)

server.close()

In [3]:
l//n_d

59993

In [9]:
header//n_h

39597